# Desafio 1

Painel COVID - https://covid.saude.gov.br/

Autor: Daniel Andrade

Data: 05/11/2021

3. Criar as 3 vizualizações pelo Spark com os dados enviados para o HDFS:
4. Salvar a primeira visualização como tabela Hive
5. Salvar a segunda visualização com formato parquet e compressão snappy
6. Salvar a terceira visualização em um tópico no Kafka
7. Criar a visualização pelo Spark com os dados enviados para o HDFS:
8. Salvar a visualização do exercício 6 em um tópico no Elastic
9. Criar um dashboard no Elastic para visualização dos novos dados enviados

## Visualizações da tabela Hive

Utilizada a linguagem SQL para gerar as visualizações

### Imports e configurações

In [3]:
from pyspark.sql import HiveContext

In [4]:
spark

In [5]:
sqlContext = HiveContext(sc)

### Exploração dos dados

In [6]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|       covid|
|     default|
+------------+



In [7]:
sqlContext.sql("use covid").show()

++
||
++
++



In [8]:
sqlContext.sql("show tables").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|   covid| tbl_brasil_acum|      false|
|   covid|tbl_brasil_recup|      false|
|   covid|   tbl_covid_ext|      false|
|   covid|  tbl_covid_part|      false|
+--------+----------------+-----------+



In [9]:
sqlContext.sql("select count(*) from tbl_covid_part").show()

+--------+
|count(1)|
+--------+
| 3304844|
+--------+



In [10]:
sqlContext.sql("describe tbl_covid_part").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|              regiao|   string|   null|
|              estado|   string|   null|
|           municipio|   string|   null|
|               coduf|      int|   null|
|              codmun|      int|   null|
|      codregiaosaude|      int|   null|
|     nomeregiaosaude|   string|   null|
|                data|     date|   null|
|           semanaepi|      int|   null|
|    populacaotcu2019|      int|   null|
|      casosacumulado|      int|   null|
|          casosnovos|      int|   null|
|     obitosacumulado|      int|   null|
|         obitosnovos|      int|   null|
|    recuperadosnovos|      int|   null|
|emacompanhamenton...|      int|   null|
|interior_metropol...|      int|   null|
+--------------------+---------+-------+



In [11]:
sqlContext.sql("select count(*) from tbl_covid_part where data >= '2021-01-01'").show()

+--------+
|count(1)|
+--------+
| 1730652|
+--------+



In [12]:
sqlContext.sql("select municipio, data from tbl_covid_part where municipio='Santos' and data >= '2021-01-01'").show(5)

+---------+----------+
|municipio|      data|
+---------+----------+
|   Santos|2021-01-01|
|   Santos|2021-01-02|
|   Santos|2021-01-03|
|   Santos|2021-01-04|
|   Santos|2021-01-05|
+---------+----------+
only showing top 5 rows



In [13]:
sqlContext.sql("select regiao, data, casosacumulado from tbl_covid_part where regiao = 'Brasil' and data >= '2021-01-01'").show(5)

+------+----------+--------------+
|regiao|      data|casosacumulado|
+------+----------+--------------+
|Brasil|2021-01-01|       7700578|
|Brasil|2021-01-02|       7716405|
|Brasil|2021-01-03|       7733746|
|Brasil|2021-01-04|       7753752|
|Brasil|2021-01-05|       7810400|
+------+----------+--------------+
only showing top 5 rows



In [14]:
sqlContext.sql("select max(data) from tbl_covid_part where regiao = 'Brasil'").show(5)

+----------+
| max(data)|
+----------+
|2021-11-04|
+----------+



In [15]:
sqlContext.sql("select regiao, data, casosacumulado from tbl_covid_part where regiao = 'Brasil' and data >= '2021-11-04'").show()

+------+----------+--------------+
|regiao|      data|casosacumulado|
+------+----------+--------------+
|Brasil|2021-11-04|      21849137|
+------+----------+--------------+



In [16]:
sqlContext.sql("select regiao, sum(casosacumulado) from tbl_covid_part where regiao <> 'Brasil' and regiao = 'Sudeste' and trim(municipio) <> ''  and data = '2021-11-04' group by regiao").show()

+-------+-------------------+
| regiao|sum(casosacumulado)|
+-------+-------------------+
|Sudeste|            8520937|
+-------+-------------------+



In [17]:
sqlContext.sql("select regiao, sum(casosacumulado) from tbl_covid_part where regiao <> 'Brasil' and trim(municipio) <> '' and data = '2021-11-04' group by regiao").show()

+------------+-------------------+
|      regiao|sum(casosacumulado)|
+------------+-------------------+
|    Nordeste|            4837356|
|         Sul|            4242401|
|     Sudeste|            8520937|
|Centro-Oeste|            2344394|
|       Norte|            1863191|
+------------+-------------------+



In [18]:
sqlContext.sql("select sum(casosacumulado) from tbl_covid_part where regiao <> 'Brasil' and trim(municipio) <> '' and data = '2021-11-04'").show()

+-------------------+
|sum(casosacumulado)|
+-------------------+
|           21808279|
+-------------------+



In [19]:
sqlContext.sql("select sum(casosnovos) from tbl_covid_part where regiao = 'Brasil' and data = '2021-11-04'").show()

+---------------+
|sum(casosnovos)|
+---------------+
|          13352|
+---------------+



In [20]:
sqlContext.sql("select regiao, sum(casosnovos) from tbl_covid_part where regiao <> 'Brasil' and trim(municipio) <> '' and data = '2021-11-04' group by regiao").show()

+------------+---------------+
|      regiao|sum(casosnovos)|
+------------+---------------+
|    Nordeste|           1979|
|         Sul|           3350|
|     Sudeste|           5102|
|Centro-Oeste|           1902|
|       Norte|           1010|
+------------+---------------+



In [21]:
sqlContext.sql("select sum(obitosacumulado) from tbl_covid_part where regiao = 'Brasil' and data = '2021-11-04'").show()

+--------------------+
|sum(obitosacumulado)|
+--------------------+
|              608671|
+--------------------+



In [22]:
sqlContext.sql("select sum(obitosnovos) from tbl_covid_part where regiao = 'Brasil' and data = '2021-11-04'").show()

+----------------+
|sum(obitosnovos)|
+----------------+
|             436|
+----------------+



In [23]:
sqlContext.sql("select sum(recuperadosnovos) from tbl_covid_part where regiao = 'Brasil' and data = '2021-11-04'").show()

+---------------------+
|sum(recuperadosnovos)|
+---------------------+
|             21047296|
+---------------------+



## Visulização 1 (Recuperados)

In [24]:
sqlContext.sql("select regiao, data, populacaotcu2019, recuperadosnovos, emacompanhamentonovos from tbl_covid_part where regiao = 'Brasil' and data >= '2021-11-04'").show()

+------+----------+----------------+----------------+---------------------+
|regiao|      data|populacaotcu2019|recuperadosnovos|emacompanhamentonovos|
+------+----------+----------------+----------------+---------------------+
|Brasil|2021-11-04|       210147125|        21047296|               193170|
+------+----------+----------------+----------------+---------------------+



In [25]:
df_brasil_recup = sqlContext.sql("select regiao, data, populacaotcu2019, recuperadosnovos, emacompanhamentonovos from tbl_covid_part where regiao = 'Brasil' and data >= '2021-11-04'")

### Salva como Hive Table

In [32]:
df_brasil_recup.createOrReplaceTempView("tbl_brasil_temp_recup")

In [33]:
sqlContext.sql("create table if not exists tbl_brasil_recup as select * from tbl_brasil_temp_recup")

DataFrame[]

### Salva como Parquet

In [42]:
#!pip install fabric3

In [43]:
from fabric.api import local
from fabric.context_managers import settings

file_exists = False
with settings(warn_only=True):
    result = local('hadoop fs -stat hdfs://namenode:8020/user/root/brasil.recup.parquet', capture=True)
    file_exists = result.succeeded

[localhost] local: hadoop fs -stat hdfs://namenode:8020/user/root/brasil.recup.parquet


In [44]:
file_exists

True

In [45]:
if not file_exists:
  df_brasil_recup.write.parquet("brasil.recup.parquet")

## Visualização 2 (Confirmados)

In [46]:
sqlContext.sql("select regiao, data, populacaotcu2019, casosacumulado, (casosacumulado*100.000)/populacaotcu2019 incidencia, casosnovos from tbl_covid_part where regiao = 'Brasil' and data >= '2021-11-04'").show()

+------+----------+----------------+--------------+-----------------+----------+
|regiao|      data|populacaotcu2019|casosacumulado|       incidencia|casosnovos|
+------+----------+----------------+--------------+-----------------+----------+
|Brasil|2021-11-04|       210147125|      21849137|10.39706681687889|     13352|
+------+----------+----------------+--------------+-----------------+----------+



### Coeficiente de Incidência de COVID-19

Método de cálculo

Número de casos confirmados de COVID-19 em residentes X 100.000 / População* total residente no período determinado.

In [47]:
df_brasil = sqlContext.sql("select regiao, data, populacaotcu2019, casosacumulado, (casosacumulado*100.000)/populacaotcu2019 incidencia, casosnovos from tbl_covid_part where regiao = 'Brasil' and data >= '2021-11-04'")

### Salva como Hive Table

In [48]:
df_brasil.createOrReplaceTempView("tbl_brasil_temp")

In [49]:
sqlContext.sql("create table if not exists tbl_brasil_acum as select * from tbl_brasil_temp")

DataFrame[]

### Salva como Parquet

In [51]:
from fabric.api import local
from fabric.context_managers import settings

file_exists = False
with settings(warn_only=True):
    result = local('hadoop fs -stat hdfs://namenode:8020/user/root/brasil.acum.parquet', capture=True)
    file_exists = result.succeeded

[localhost] local: hadoop fs -stat hdfs://namenode:8020/user/root/brasil.acum.parquet


In [52]:
file_exists

True

In [53]:
if not file_exists:
  df_brasil.write.parquet("brasil.acum.parquet")

## Visualização 3 (Óbitos)

In [54]:
sqlContext.sql("select regiao, data, populacaotcu2019, obitosacumulado, (obitosacumulado*100.000)/populacaotcu2019 coeficiente, (obitosacumulado*100.000)/casosacumulado taxa, obitosnovos from tbl_covid_part where regiao = 'Brasil' and data = '2021-11-04'").show()

+------+----------+----------------+---------------+----------------+----------------+-----------+
|regiao|      data|populacaotcu2019|obitosacumulado|     coeficiente|            taxa|obitosnovos|
+------+----------+----------------+---------------+----------------+----------------+-----------+
|Brasil|2021-11-04|       210147125|         608671|0.28964041263948|2.78578966299676|        436|
+------+----------+----------------+---------------+----------------+----------------+-----------+



#### Coeficiente de Mortalidade por COVID-19

Método de cálculo

Número de óbitos confirmados de COVID-19 em residentes X 100.000 / População* total residente no período determinado.

#### Taxa de Letalidade por COVID-19

Método de cálculo

Número de óbitos confirmados de COVID-19 em determinada área e período X 100 / Número de casos confirmados de COVID-19 em determinada área e período.

## Visualização 4 (Região)

In [55]:
sqlContext.sql("select regiao, sum(casosacumulado) casos, sum(obitosacumulado) obitos, data, sum(populacaotcu2019) from tbl_covid_part where regiao <> 'Brasil' and trim(municipio) <> '' and data = '2021-11-04' group by regiao, data").show()

+------------+-------+------+----------+---------------------+
|      regiao|  casos|obitos|      data|sum(populacaotcu2019)|
+------------+-------+------+----------+---------------------+
|    Nordeste|4837356|117870|2021-11-04|             57071654|
|     Sudeste|8520937|288634|2021-11-04|             88371433|
|       Norte|1863191| 46836|2021-11-04|             18430980|
|         Sul|4242401| 95654|2021-11-04|             29975984|
|Centro-Oeste|2344394| 58515|2021-11-04|             16297074|
+------------+-------+------+----------+---------------------+



In [56]:
sqlContext.sql("select regiao, casos, obitos, (casos*100.000)/populacao incidencia, (obitos*100.000)/populacao mortalidade, data from (select regiao, sum(casosacumulado) casos, sum(obitosacumulado) obitos, data, sum(populacaotcu2019) populacao from tbl_covid_part where regiao <> 'Brasil' and trim(municipio) <> '' and data = '2021-11-04' group by regiao, data) x").show()

+------------+-------+------+-----------------+----------------+----------+
|      regiao|  casos|obitos|       incidencia|     mortalidade|      data|
+------------+-------+------+-----------------+----------------+----------+
|    Nordeste|4837356|117870| 8.47593448053915|0.20652984754919|2021-11-04|
|     Sudeste|8520937|288634| 9.64218493548701|0.32661459727602|2021-11-04|
|       Norte|1863191| 46836|10.10901753460749|0.25411562488810|2021-11-04|
|         Sul|4242401| 95654|14.15266634783365|0.31910211854930|2021-11-04|
|Centro-Oeste|2344394| 58515|14.38536758193526|0.35905218323240|2021-11-04|
+------------+-------+------+-----------------+----------------+----------+

